In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

##### Torch Sample Net Class
Pytorch 에서는 Neural Network Module 클래스를 제공해준다. Module은 파라미터 분석, GPU, Exporting, Loading 등의 편리한 기능을 제공한다.

특히 forward 부분만 제공해도 back propagation 부분을 자동으로 제공해주는데, Autograd 기능을 기본적으로 사용하기 때문에 사용하기 쉽다.

**참고자료**
* https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        self.fc1 = nn.Linear(16 * 6 * 6, 128)
        self.fc2 = nn.Linear(128, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x)) # flatten
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        print("x:", x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features

net = Net()

In [10]:
params = list(net.parameters())

In [15]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

x: tensor([[ 0.0437, -0.1248,  0.0678,  0.1429,  0.0958,  0.0628,  0.0539, -0.0716,
         -0.0768, -0.0179]], grad_fn=<AddmmBackward>)
tensor([[ 0.0437, -0.1248,  0.0678,  0.1429,  0.0958,  0.0628,  0.0539, -0.0716,
         -0.0768, -0.0179]], grad_fn=<AddmmBackward>)


In [128]:
# backward opartion.
net.zero_grad()
out.backward(torch.randn(1, 10))

In [155]:
# Loss 
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)

print(loss)
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

tensor(1.4171, grad_fn=<MseLossBackward>)


In [156]:
# Backpropagation
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.weight.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.weight.grad)

conv1.bias.grad before backward
tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])
conv1.bias.grad after backward
tensor([[[[-0.0072,  0.0018, -0.0066],
          [ 0.0045,  0.0120, -0.0136],
          [-0.0083,  0.0027,  0.0231]]],


        [[[ 0.0267,  0.0181, -0.0174],
          [-0.0102, -0.0016, -0.0152],
          [-0.0170,  0.0095, -0.0092]]],


        [[[ 0.0130,  0.0023, -0.0120],
          [-0.0120,  0.0069,  0.0037],
          [-0.0149, -0.0054, -0.0069]]],


        [[[ 0.0009,  0.0162, -0.0086],
          [ 0.0080,  0.0076,  0.0048],
          [-0.0159,  0.0240,  0.0225]]],
